In [1]:
import tensorflow as tf
import keras 
from keras import layers
import numpy as np
from numba import cuda
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import re
from keras.utils import img_to_array
import gc

In [2]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
color_img = []
path = 'E:/TER//landscape Images/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
        if i == '7200.jpg':
            break
        else:    
            img = cv2.imread(path + '/'+i,1)
            # open cv reads images in BGR format so we have to convert it to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #resizing image
            img = cv2.resize(img, (SIZE, SIZE))
            img = img.astype('float32') / 255.0
            color_img.append(img_to_array(img))
color_dataset=tf.data.Dataset.from_tensor_slices(np.array(color_img[0:2000])).batch(1)
gc.collect()

gray_img = []
path = 'E:/TER//landscape Images/gray'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
         if i == '7200.jpg':
            break
         else: 
            img = cv2.imread(path + '/'+i,1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #resizing image
            img = cv2.resize(img, (SIZE, SIZE))
            img = img.astype('float32') / 255.0
            gray_img.append(img_to_array(img))
gray_dataset=tf.data.Dataset.from_tensor_slices(np.array(gray_img[0:2000])).batch(1)
gc.collect()

100%|██████████| 7129/7129 [00:09<00:00, 759.30it/s] 


9

In [3]:
generator = keras.models.load_model('E:/TER/model/generator_rgb.h5')
#f = open('E:/TER/landscape Images/generator.txt', 'w')
#清除文件中的内容
#f.truncate()

In [4]:
def generate_images(model, test_input, tar, n):
  prediction = model(test_input, training=False)
  #MAE = sum(tf.keras.losses.mean_absolute_error(tar[0], prediction[0]))
  plt.figure(figsize=(15,5))
  prediction = np.clip(np.array(prediction),0,1)
  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']
  #strtemp = ("Mean Absolute Error of"+str(n)+"is"+str(MAE.numpy())+"\n")
  #f.write(strtemp)  
  M = tf.keras.losses.MeanSquaredError()
  MSE = M(tar[0], prediction[0])
  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i])
    plt.axis('off')
  plt.suptitle("MSE: "+str(MSE.numpy()),fontsize=20)
  plt.savefig('E:/TER/Result_pix2pix_landscape_RGB/plot/'+str(n)+".png")
  plt.imsave('E:/TER/Result_pix2pix_landscape_RGB/output/'+str(n)+".png", prediction[0])
  plt.close()
  #保存prediction为图片
  #cv2.imwrite('E:/TER/landscape Images/output/prediction'+str(n)+'.jpg',prediction[0]*255)

In [5]:
with tqdm(total=7129) as pbar: 
  color_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(color_img[0:2000])).batch(1)
  gray_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(gray_img[0:2000])).batch(1)
  n = 0
  for example_input, example_target in tf.data.Dataset.zip((gray_dataset_t,color_dataset_t)):
    generate_images(generator, example_input, example_target,n)
    n = n + 1
    pbar.update(1)
  color_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(color_img[2000:4000])).batch(1)
  gray_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(gray_img[2000:4000])).batch(1)
  for example_input, example_target in tf.data.Dataset.zip((gray_dataset_t,color_dataset_t)):
    generate_images(generator, example_input, example_target,n)
    n = n + 1
    pbar.update(1)
  color_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(color_img[4000:6000])).batch(1)
  gray_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(gray_img[4000:6000])).batch(1)
  for example_input, example_target in tf.data.Dataset.zip((gray_dataset_t,color_dataset_t)):
    generate_images(generator, example_input, example_target,n)
    n = n + 1
    pbar.update(1)
  color_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(color_img[6000:7129])).batch(1)
  gray_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(gray_img[6000:7129])).batch(1)
  for example_input, example_target in tf.data.Dataset.zip((gray_dataset_t,color_dataset_t)):
    generate_images(generator, example_input, example_target,n)
    n = n + 1
    pbar.update(1)
#f.close()

100%|██████████| 7129/7129 [30:35<00:00,  3.88it/s]  
